In [2]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# Iterators, generators and itertools

In [4]:
for i in range(5): print(i, end=" ")
print()
for i in (0, 1, 2, 3, 4): print(i, end=" ")
print()
for i in {0, 1, 2, 3, 4}: print(i, end=" ")
list(map(type, (range(5), (0, 1, 2, 3, 4), {0, 1, 2, 3, 4})))

0 1 2 3 4 
0 1 2 3 4 
0 1 2 3 4 

[range, tuple, set]

In [5]:
range(5).__sizeof__(), (0, 1, 2, 3, 4).__sizeof__(), {0, 1, 2, 3, 4}.__sizeof__()

(48, 64, 712)

In [6]:
mil = range(10**6)
mil.__sizeof__(), tuple(mil).__sizeof__()

(48, 8000024)

## Collections, sequences and containers

<img src="https://miro.medium.com/max/1200/1*X3GmUh7dqAMJLM5KwGBKYQ.png">

## Iterable

In [7]:
hasattr(range(5), "__iter__"), hasattr(tuple(), "__iter__"), hasattr(set(), "__iter__")

(True, True, True)

```python
iterable.__iter__() -> iterator
```

## Iterator protocol

- `__iter__` - return the iterator object itself
- `__next__` - return the next item from the iterator

Return the next item from the container. If there are no further items, raise the `StopIteration` exception. Once an iterator’s `__next__` method raises `StopIteration`, it must continue to do so on subsequent calls.

In [8]:
for i in range(5):
    print(i, end=" ")
print()

iterator = iter(range(5))
while True:
    try:
        i = next(iterator)
        print(i, end=" ")
    except StopIteration:
        break

0 1 2 3 4 
0 1 2 3 4 

In [9]:
type(range(5)), type(iterator)

(range, range_iterator)

In [11]:
from collections import namedtuple
from typing import Iterator


Page = namedtuple("Page", ["text", "number"])


class Book:
    def __init__(self) -> None:
        self.pages = []

    def add_page(self, text: str) -> None:
        self.pages.append(
            Page(text, number=len(self.pages) + 1)
        )
        
    def __iter__(self) -> Iterator[Page]:
        return BookIter(self)
    
class BookIter:
    def __init__(self, book: Book) -> None:
        self.pages = book.pages
        self._cursor = -1
        
    def __iter__(self) -> "BookIter":
        return self
    
    def __next__(self) -> Page:
        self._cursor += 1
        if len(self.pages) > self._cursor:
            return self.pages[self._cursor]
        raise StopIteration

In [12]:
book = Book()
for i in range(1, 5): 
    book.add_page(f"page_{i}")

for page in book:
    print(page)

Page(text='page_1', number=1)
Page(text='page_2', number=2)
Page(text='page_3', number=3)
Page(text='page_4', number=4)


In [13]:
type(book), type(iter(book))

(__main__.Book, __main__.BookIter)

## Why do we need BookIter?

In [14]:
class LazyBook(Book):
    def __iter__(self) -> Iterator[Page]:
        return iter(self.pages)

In [15]:
lazy_book = LazyBook()
for i in range(1, 5): 
    lazy_book.add_page(f"page_{i}")

for page in lazy_book:
    print(page)

Page(text='page_1', number=1)
Page(text='page_2', number=2)
Page(text='page_3', number=3)
Page(text='page_4', number=4)


In [16]:
type(lazy_book), type(iter(lazy_book))

(__main__.LazyBook, list_iterator)

In [36]:
class PurchasableBook(Book):
    def __init__(self, purchased: bool = False) -> None:
        self.purchased = purchased
        super().__init__()
        
    def __iter__(self) -> "PurchasableBookIter":
        return PurchasableBookIter(self)
    

class BookIter:
    def __init__(self, book: Book) -> None:
        self.pages = book.pages
        self.book = book
        self._cursor = 0  # self._cursor = -1
        
    def __iter__(self) -> "BookIter":
        return self
    
    def __next__(self) -> Page:
        if len(self.pages) > self._cursor:     # self._cursor += 1
            result = self.pages[self._cursor]  # if len(self.pages) > self._cursor:
            self._cursor += 1                  #     return self.pages[self._cursor]
            return result
        raise StopIteration                    # raise StopIteration
        
        
class PurchasableBookIter(BookIter):
    def __init__(self, book: Book):
        self.purchased = book.purchased
        super().__init__(book)
    
    def __next__(self) -> Page:
        if not self.purchased and self._cursor > 0:
            print("Buy the book to view next pages!")
            raise StopIteration
        return super().__next__()

In [37]:
purchased_book = PurchasableBook()
for i in range(1, 5): 
    purchased_book.add_page(f"page_{i}")

    
for page in purchased_book:
    print(page)

Page(text='page_1', number=1)
Buy the book to view next pages!


In [38]:
it = iter(purchased_book)

In [39]:
for page in it:
    print(page)

Page(text='page_1', number=1)
Buy the book to view next pages!


In [40]:
purchased_book.purchased = True
for page in it:
    print(page)

Buy the book to view next pages!


In [41]:
purchased_book.purchased = True
for page in purchased_book:
    print(page)

Page(text='page_1', number=1)
Page(text='page_2', number=2)
Page(text='page_3', number=3)
Page(text='page_4', number=4)


## Is PurchasableBookIter fully match the iterator protocol?

<h2 align=center>Quiz time</h2>

$$
iterators \supset iterable
$$
<center>or</center>
$$
iterators \subset iterable
$$

## Recup

What should an iterator do?

1. Track the current state
1. Know how to return next element
1. ?????
1. <strike>PROFIT</strike> `StopIteration`

Do we realy need a collection for an iterator?

Do we realy need to stop?

In [42]:
class RecurrentSequence:
    def __init__(self, a_1: int, a_2: int) -> None:
        self.a_1 = a_1
        self.a_2 = a_2
    
    def __iter__(self) -> Iterator[int]:
        return RecurrentSequenceIterator(self.a_1, self.a_2)

    
class RecurrentSequenceIterator:
    def __init__(self, a_1: int, a_2: int) -> None:
        self.a_1 = a_1
        self.a_2 = a_2
        
    def __iter__(self) -> Iterator[int]:
        return self
    
    def __next__(self) -> int:
        result = self.a_1
        self.a_1, self.a_2 = self.a_2, self.a_1 + self.a_2
        return result

In [49]:
fib = RecurrentSequence(1, 1)
 
for i, f in zip(range(1, 20), fib):
    print(f"{i} - {f}", end="; ")
    
print()
for i, f in zip(range(1, 20), fib):
    print(f"{i} - {f}", end="; ")

1 - 1; 2 - 1; 3 - 2; 4 - 3; 5 - 5; 6 - 8; 7 - 13; 8 - 21; 9 - 34; 10 - 55; 11 - 89; 12 - 144; 13 - 233; 14 - 377; 15 - 610; 16 - 987; 17 - 1597; 18 - 2584; 19 - 4181; 
1 - 1; 2 - 1; 3 - 2; 4 - 3; 5 - 5; 6 - 8; 7 - 13; 8 - 21; 9 - 34; 10 - 55; 11 - 89; 12 - 144; 13 - 233; 14 - 377; 15 - 610; 16 - 987; 17 - 1597; 18 - 2584; 19 - 4181; 

In [46]:
fib_iter = iter(fib)

for i, f in zip(range(1, 10), fib_iter):
    print(f"{i} - {f}", end="; ")
print()

for i, f in zip(range(1, 10), fib_iter):
    print(f"{i} - {f}", end="; ")

1 - 1; 2 - 1; 3 - 2; 4 - 3; 5 - 5; 6 - 8; 7 - 13; 8 - 21; 9 - 34; 
1 - 55; 2 - 89; 3 - 144; 4 - 233; 5 - 377; 6 - 610; 7 - 987; 8 - 1597; 9 - 2584; 

In [50]:
fib.__sizeof__(), fib_iter.__sizeof__()

(32, 32)

## Any side effects?

- one can exhauste an iterator:

In [52]:
iterator = iter([1, 2, 3, 4])
print(sum(iterator))
print(sum(iterator))

not_iterator = [1, 2, 3, 4]
print(sum(not_iterator))
print(sum(not_iterator))

10
0
10
10


- an iterable has `__contains__` method

In [63]:
print(list(book))
Page("page_2", 2) in book, Page("page_2", 2) in book, Page("page_5", 5) in book, 3 in book

[Page(text='page_1', number=1), Page(text='page_2', number=2), Page(text='page_3', number=3), Page(text='page_4', number=4)]


(True, True, False, False)

In [59]:
iterator = iter(book)
Page("page_2", 2) in iterator, Page("page_2", 2) in iterator

(True, False)

In [64]:
5 in fib
# 6 in fib

True

## Iterables with `__getitem__`

In [65]:
class HiddenList:
    def __init__(self, lst):
        self._lst = lst


h_list = HiddenList([1, 2, 3])
iter(h_list)

TypeError: 'HiddenList' object is not iterable

In [79]:
class IterableHiddenList(HiddenList):
    def __getitem__(self, index):
        print(f"Index: {index}")
        return self._lst[index]
    

ih_list = IterableHiddenList([1, 2, 3])
iter(ih_list)

# for i in ih_list:
#     print(i)
#     pass
    
# 5 in ih_list

How might it work?

In [67]:
print(dir(IterableHiddenList))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


In [68]:
print(dir(RecurrentSequence))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


```python
print(index)
```

## Any questions so far?

# GENERATORS

Are generators iterators?

In [80]:
def gen():
    yield 1

In [82]:
print(dir(gen()))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


Are generators iterators?

\- Yes*

In [83]:
gen, gen(), (1 for _ in [])

(<function __main__.gen()>,
 <generator object gen at 0x7f31243fadd0>,
 <generator object <genexpr> at 0x7f3124209ed0>)

- **Generator functions** - a function or method which uses the yield statement
- **Generator iterator** - an object created by a generator function
- **Generator expression** -an expression that returns an iterator

In [84]:
def recurrent_sequence(a1: int, a2: int):
    while True:
        yield a1
        a1, a2 = a2, a1 + a2
        
fib = recurrent_sequence(0, 1)

for i, f in zip(range(1, 20), fib):
    print(f"{i} - {f}", end="; ")

1 - 0; 2 - 1; 3 - 1; 4 - 2; 5 - 3; 6 - 5; 7 - 8; 8 - 13; 9 - 21; 10 - 34; 11 - 55; 12 - 89; 13 - 144; 14 - 233; 15 - 377; 16 - 610; 17 - 987; 18 - 1597; 19 - 2584; 

## Generators and `return`

In [85]:
def gen():
    yield 1
    return 2

In [86]:
g = gen()
next(g)

1

In [87]:
next(g)

StopIteration: 2

What will happen here?

In [95]:
def gen():
    return 2
    yield 1
    
gen().__next__()

StopIteration: 2

In [96]:
try:
    next(gen())
except StopIteration as e:
    print(e.value)

2


## Preserving operations order

In [101]:
def do_in_order():
    x = 1
    print(f"Do first, {x}")
    yield
    x += 1
    print(f"Do second, {x}")
    yield
    x *= x
    print(f"Do third, {x}")

gen = do_in_order()
next(gen)
next(gen)
next(gen, "Stop")

Do first, 1
Do second, 2
Do third, 4


'Stop'

## Send

In [104]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    y = yield
    print(f"Do second, {y}")
    z = yield 42
    print(f"Do third, {z}")

gen = do_in_order_2()
for _ in gen:
    print(f"step {_}")

Do first, 1
step None
Do second, None
step 42
Do third, None


In [105]:
gen = do_in_order_2()
next(gen)
next(gen)
next(gen, "Stop")

Do first, 1
Do second, None
Do third, None


'Stop'

## Send

In [110]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    x = yield "123"
    print(f"Do second, {x}")
    x = yield 42
    print(f"Do third, {x}")

gen = do_in_order_2()

In [111]:
gen.send(None)

Do first, 1


'123'

In [108]:
gen.send("Hello")

Do second, Hello


42

In [109]:
try:
    gen.send("World")
except StopIteration:
    print("I'm out!")

Do third, World
I'm out!


## Throw

In [112]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

gen = g()
next(gen)

42

In [113]:
gen.throw(NotImplementedError, "Exception text")

NotImplementedError('Exception text')

In [114]:
gen.throw(NotImplementedError, "Exception text returns")

NotImplementedError: Exception text returns

## Close

In [115]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    x = yield
    print(f"Do second, {x}")
    x = yield 42
    print(f"Do third, {x}")

gen = do_in_order_2()

In [116]:
gen.send(None)

Do first, 1


In [117]:
gen.close()

In [118]:
gen.send(None)

StopIteration: 

In [119]:
gen.throw(NotImplementedError, "Exception text")

NotImplementedError: Exception text

How did `close` works?

In [120]:
BaseException.__subclasses__()

[Exception, GeneratorExit, SystemExit, KeyboardInterrupt]

In [121]:
def tricky_gen():
    yield "first"
    try:
        yield "second"
    finally:
        yield "from finally"

gen = tricky_gen()

In [122]:
for i in gen:
    print(i, end=" ")

first second from finally 

In [125]:
gen = tricky_gen()

next(gen), next(gen)

('first', 'second')

In [129]:
gen.close()

https://amir.rachum.com/blog/2017/03/03/generator-cleanup/

## yield from

- pass the execution to another generator
- pass `send` and `throw`

In [132]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item

In [133]:
list(chain(
    [1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter", {("a",), ("b",)}
))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r', ('a',), ('b',)]

Can we do better?

In [134]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable
    return "Stop"

In [135]:
list(chain(
    [1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter", {("a",), ("b",)}
))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r', ('a',), ('b',)]

## Example: Sieve of Eratosthenes

In mathematics, the sieve of Eratosthenes is an ancient algorithm for finding all prime numbers up to any given limit. 

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Sieve_of_Eratosthenes_animation.gif">

1. Create a list of consecutive integers from 2 through n: (2, 3, 4, ..., n).
1. Initially, let p equal 2, the smallest prime number.
1. Enumerate the multiples of p by counting in increments of p from 2p to n, and mark them in the list (these will be 2p, 3p, 4p, ...; the p itself should not be marked).
1. Find the smallest number in the list greater than p that is not marked. If there was no such number, stop. Otherwise, let p now equal this new number (which is the next prime), and repeat from step 3.
1. When the algorithm terminates, the numbers remaining not marked in the list are all the primes below n.

Step 1: generate all natural numbers:

In [138]:
def natural_numbers(start=1):
    while True:
        yield start
        start += 1
        
for _, number in zip(range(10), natural_numbers(1)):
    print(number, end=" ")

1 2 3 4 5 6 7 8 9 10 

Step 2: ~draw the rest of the owl~

In [150]:
s = sieve(natural_numbers(2))

In [143]:
def sieve(numbers):
    prime = next(numbers)
    yield prime
    yield from sieve(p for p in numbers if p % prime != 0)
    
for _, prime in zip(range(10), sieve(natural_numbers(2))):
    print(f"{_} - {prime}")

0 - 2
1 - 3
2 - 5
3 - 7
4 - 11
5 - 13
6 - 17
7 - 19
8 - 23
9 - 29


# Lazy evaluations

In [558]:
n = 10**7

In [561]:
%%memit 
sum([i for i in range(n)])

peak memory: 475.96 MiB, increment: 382.55 MiB


In [562]:
%%memit 
sum(i for i in range(n))

peak memory: 94.05 MiB, increment: 0.00 MiB


In [563]:
%%memit
sum(list(map(lambda x: x**2, [i for i in range(n)])))

peak memory: 862.08 MiB, increment: 768.04 MiB


In [564]:
%%memit
sum(map(lambda x: x**2, [i for i in range(n)]))

peak memory: 476.36 MiB, increment: 382.31 MiB


In [565]:
%%memit
sum(map(lambda x: x**2, (i for i in range(n))))

peak memory: 94.14 MiB, increment: 0.00 MiB


# Context managers

- `__enter__(self)`
- `__exit__(self, exception_type, exception_value, traceback)`

Patterns:
- acquisition/release of the resource
- doing something in different context

In [571]:
import os


class cd:
    def __init__(self, path):
        self.path = path
        
    def __enter__(self):
        self.cwd = os.getcwd()
        os.chdir(self.path)
        
    def __exit__(self, *args):
        os.chdir(self.cwd)

        
print(os.getcwd())
with cd(".."):
    print(os.getcwd())
print(os.getcwd())

/
/
/


In [574]:
from contextlib import contextmanager


@contextmanager
def cd_gen(path):
    # <__init__>
    cwd = os.getcwd()
    # </__init__>
    try:
        # <__enter__>
        os.chdir(path)
        # </__enter__>
        yield
    finally:
        # <__exit__>
        os.chdir(cwd)
        # </__exit__>
        
print(os.getcwd())
with cd_gen("/home"):
    print(os.getcwd())
print(os.getcwd())

/
/home
/


# `itertools`

This module implements a number of iterator building blocks inspired by constructs from APL, Haskell, and SML. Each has been recast in a form suitable for Python.

The module standardizes a core set of fast, memory efficient tools that are useful by themselves or in combination. Together, they form an “iterator algebra” making it possible to construct specialized tools succinctly and efficiently in pure Python.

In [576]:
from itertools import islice

In [577]:
def take(iterable, n=10):
    return list(islice(iterable, n))

In [580]:
take(fib, 5)

[514229, 832040, 1346269, 2178309, 3524578]

In [581]:
from itertools import count, repeat, cycle

In [582]:
take(count())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [583]:
take(repeat([1, 2]), 3)

[[1, 2], [1, 2], [1, 2]]

In [584]:
take(cycle([1, 2]), 11)

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]

In [585]:
from itertools import dropwhile, takewhile

In [586]:
list(dropwhile(lambda x: x < 3, range(6)))

[3, 4, 5]

In [587]:
list(takewhile(lambda x: x < 3, range(6)))

[0, 1, 2]

In [588]:
from itertools import chain

In [589]:
list(chain(
    [1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter", {("a",), ("b",)}
))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r', ('a',), ('b',)]

In [590]:
collection = [[1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter", {("a",), ("b",)}]

In [591]:
list(chain(*collection))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r', ('a',), ('b',)]

In [592]:
list(chain.from_iterable(collection))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r', ('a',), ('b',)]

What is the difference?

In [593]:
from itertools import tee

In [594]:
iterator = iter(range(4))
a = b = iterator

list(a), list(b)

([0, 1, 2, 3], [])

In [599]:
iterator = range(4)
a, b = tee(iterator, 2)

list(iterator)

list(a), list(b)

([0, 1, 2, 3], [0, 1, 2, 3])

In [596]:
iterator = iter(range(4))
a, b = tee(iterator, 2)

list(iterator)

list(a), list(b)

([], [])

## Combinatoric iterators

In [600]:
from itertools import product

In [602]:
list(product("12", repeat=3))

[('1', '1', '1'),
 ('1', '1', '2'),
 ('1', '2', '1'),
 ('1', '2', '2'),
 ('2', '1', '1'),
 ('2', '1', '2'),
 ('2', '2', '1'),
 ('2', '2', '2')]

In [603]:
list(product("12", "abc"))

[('1', 'a'), ('1', 'b'), ('1', 'c'), ('2', 'a'), ('2', 'b'), ('2', 'c')]

In [ ]:
sum(1 for a, b, c, d in product(l1, l2, l3, l4) if a + b + c + d == 0)
# or
sum(a + b + c + d == 0 for a, b, c, d in product(l1, l2, l3, l4))

In [604]:
from itertools import permutations, combinations, combinations_with_replacement

In [606]:
list(permutations("BAC", 2))

[('B', 'A'), ('B', 'C'), ('A', 'B'), ('A', 'C'), ('C', 'B'), ('C', 'A')]

In [607]:
list(combinations("ABC", 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [608]:
list(combinations_with_replacement("ABC", 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

# Links

More itertools:

https://docs.python.org/3/library/itertools.html#itertools-recipes


David Beazley:

https://www.dabeaz.com/tutorials.html

# Fin.